In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup



In [10]:
search = input()

iphone 14


In [11]:
limit = int(input())

2


In [17]:
s = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service = s)
driver.get('https://www.amazon.in/')

In [18]:
try:
    un = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="twotabsearchtextbox"]')))  #//*[@id="nav-bb-search"]
    un.send_keys(search)
    time.sleep(2)
    un.send_keys(Keys.ENTER)
except TimeoutException:
    un = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="nav-bb-search"]')))  #
    un.send_keys(search)
    time.sleep(2)
    un.send_keys(Keys.ENTER)
    

#<a class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" target="_blank" href="/sspa/click?ie=UTF8&amp;spc=MToyMTE0OTc4MDA2ODA4NjAwOjE2ODE4NTQwNTg6c3BfYXRmX25leHQ6MjAwNzcyNDMxOTQxOTg6OjA6Og&amp;url=%2FApple-iPhone-14-128GB-Blue%2Fdp%2FB0BDK62PDX%2Fref%3Dsr_1_49_sspa%3Fcrid%3D32B8VMS6ZWAKD%26keywords%3Diphone%2B14%26qid%3D1681854058%26sprefix%3Diphone%2B14%252Caps%252C436%26sr%3D8-49-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfbmV4dA%26psc%3D1"><span class="a-size-medium a-color-base a-text-normal">Apple iPhone 14 (128 GB) - Blue</span> </a>

In [19]:
def link_collection():
    driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
    first_page_id = driver.current_window_handle
    review = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal'))) # a-size-base.s-underline-text
    reviews = []
    price = []
    
    for i in review:
        if '₹' not in i.text:
            reviews.append(i)
            
    for i in review:
        if '₹' in i.text:
            price.append(i.text)
    #price = []
    
    cnt = 0
    for i in price:
        if cnt%2 != 0:
            price.append(i)
        cnt+=1
    
    try:
        title1 = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'a-size-base-plus.a-color-base.a-text-normal')))
    except TimeoutException:
        title1 = []
        
    try: 
        title2 = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'a-size-medium.a-color-base.a-text-normal')))
    except TimeoutException:
        title2 = []
    
     
    main_title = title1 if len(title1) > len(title2) else title2
    

    titles = [] 
    d = {}
    prices = {}
    for i, j in zip(main_title, reviews):
        x = search.lower()
        if ('(Renewed)' not in i.text) or ('WeConnect' not in i.text) or ('Protect+' not in i.text): # or ('AppleCare' not in i.text) 
            d[j] = i.text
            titles.append(i.text)
    
    for i, j in zip(main_title, price):
        prices[i.text] = j
            
            
    time.sleep(1)
    return d, titles, prices

In [20]:
def scraping(d, limit, prices):
    #j = 1
    prod_review= {}
    ratings = {}
    link_of_product = {}
    overall_rating = {}
    for i in d:
        y = d[i]
        driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
        driver.switch_to.window(driver.window_handles[0])
        #driver.switch_to.window(first_page_id)
        i.click()
        time.sleep(2)
        c = driver.window_handles[1]
        driver.switch_to.window(c)
        link = driver.current_url
        link_of_product[y] = link
        
        
#         #current_page_id = driver.current_window_handle

#         #driver.switch_to.window(current_page_id)
#         try:
#             driver.switch_to.window(driver.window_handles[j])
#         except IndexError:
#             driver.switch_to.window(driver.window_handles[0])
#             j+=1
#             continue
            
        time.sleep(6)
        driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
        
        try:
            all_comments = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,'a-link-emphasis.a-text-bold'))) 
            all_comments.click()
            time.sleep(2)
        except TimeoutException:
            pass

        driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
        try:
            box = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'a-dropdown-prompt'))) 
            for k in box:
                if k.text == 'All formats':
                    k.click()
        except TimeoutException:
            pass

        try:
            click = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID,'format-type-dropdown_1')))
            click.click()
            time.sleep(3)
        except TimeoutException:
            pass  


        # scrapping views
        e = driver.page_source
        x= BeautifulSoup(e, 'html.parser')
        
        # scraping rating
        h = x.find_all('span', {'class' : 'a-size-medium a-color-base'})

        clean = ' '
        for g in h:
            cleaned = g.text.strip()
            if g.text.strip() != '':
                clean += cleaned[:3]
        overall_rating[y] = cleaned.strip()
        
        
        ## scraping review
        t = x.find_all('span', {'class' : 'a-size-base review-text review-text-content'})
        reviews = set()
        for l in t:
            reviews.add(l.text.strip())
            if len(reviews) == limit:
                break
                
        # star ratings
        h = x.find_all('td', {'class' : 'a-text-right a-nowrap'})
        rating= {}
        counter = 5
        for rat in h:
            rating['{} Stars'.format(counter)] = rat.text.strip()
            counter -=1
            
        ratings[y]=rating
        
        
        

        
        if len(reviews) != limit or len(reviews) < limit:
            try:
                n = 0
                while len(reviews) != limit:
                    try:
                        last = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME,'a-disabled.a-last')))
                        x = last.text
                        if x == 'Next page':
                            break
                    except (TimeoutException, StaleElementReferenceException):
                        review1 = reviews
                        next_ = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,'a-last')))
                        next_.click()
                        time.sleep(4)
                        driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
                        e = driver.page_source
                        x= BeautifulSoup(e, 'html.parser')
                        t = x.find_all('div', {'class' : 'a-size-base review-text review-text-content'})
                        for l in t:
                            reviews.add(l.text.strip())
                            if len(reviews) == limit:
                                break
                        prod_review[y] = reviews
                        if len(reviews) == len(review1):
                            break
            except TimeoutException:
                pass
        else:
            prod_review[y] = reviews       
            
        prod_review[y] = reviews
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

        

    return prod_review, ratings, link_of_product, overall_rating
    
    

In [21]:
review_of_product = []
ratings_of_product = []
link_product = []
price_product = []
overall_r = []

for i in range(3):
    d, titles, prices= link_collection()
    print(prices)
    prod, ratings, links, overall = scraping(d, limit, prices)
    review_of_product.append(prod)
    ratings_of_product.append(ratings)
    link_product.append(links)
    price_product.append(prices)
    overall_r.append(overall)
    print(ratings_of_product)
    print(link_product)
    
    driver.switch_to.window(driver.window_handles[0])
    driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
    time.sleep(2)
    next_ = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME,'s-pagination-item.s-pagination-next.s-pagination-button.s-pagination-separator')))
    next_.click()
    
    
    
    

{'Apple iPhone 14 (128 GB) - Blue': '₹66,999 M.R.P:\n₹79,900', 'Apple iPhone 14 Plus (128 GB) - (Product) RED': '₹75,999 M.R.P:\n₹89,900', 'Apple iPhone 14 (128 GB) - Midnight': '₹66,999 M.R.P:\n₹79,900', 'Apple iPhone 14 (128 GB) - Starlight': '₹67,999 M.R.P:\n₹79,900', 'Apple iPhone 14 (128 GB) - Purple': '₹67,999 M.R.P:\n₹79,900', 'Apple iPhone 14 (128 GB) - (Product) RED': '₹67,999 M.R.P:\n₹79,900', 'Apple iPhone 14 (256 GB) - Purple': '₹79,999 M.R.P:\n₹89,900', 'Apple iPhone 14 (128 GB) - Yellow': '₹69,999 M.R.P:\n₹79,900', 'Apple iPhone 14 (256 GB) - Midnight': '₹79,999 M.R.P:\n₹89,900', 'Apple iPhone 14 (256 GB) - Yellow': '₹79,999 M.R.P:\n₹89,900', 'Apple iPhone 14 (256 GB) - Starlight': '₹79,999 M.R.P:\n₹89,900', 'Apple iPhone 14 (256 GB) - Blue': '₹79,999 M.R.P:\n₹89,900', 'Apple iPhone 14 (256 GB) - (Product) RED': '₹79,999 M.R.P:\n₹89,900', 'Apple iPhone 14 (512 GB) - Midnight': '₹97,999 M.R.P:\n₹1,09,900', 'Apple iPhone 14 (512 GB) - Starlight': '₹97,999 M.R.P:\n₹1,09,900'

KeyboardInterrupt: 

['Apple iPhone 14 (128 GB) - Blue',
 'Apple iPhone 14 Plus (128 GB) - (Product) RED',
 'Apple iPhone 14 (128 GB) - Midnight',
 'Apple iPhone 14 (128 GB) - Starlight',
 'Apple iPhone 14 (128 GB) - Blue',
 'Apple iPhone 14 (128 GB) - Purple',
 'Apple iPhone 14 (128 GB) - (Product) RED',
 'Apple iPhone 14 (256 GB) - Purple',
 'Apple iPhone 14 (128 GB) - Yellow',
 'Apple iPhone 14 (256 GB) - Midnight',
 'Apple iPhone 14 (256 GB) - Yellow',
 'Apple iPhone 14 (256 GB) - Starlight',
 'Apple iPhone 14 (256 GB) - Blue',
 'Apple iPhone 14 (256 GB) - (Product) RED',
 'Apple iPhone 14 (512 GB) - Midnight',
 'Apple iPhone 14 (512 GB) - Starlight',
 'Apple iPhone 14 (512 GB) - Blue',
 'Apple iPhone 14 (512 GB) - Purple']

In [113]:
import pandas as pd
count = 0
length = 0
df = pd.DataFrame({'Product Name': [], 'Review': [], 'Link' : [], '5 Stars' :[], '4 Stars': [], '3 Stars': [], '2 Stars' :[], '1 Stars': [], 'Price':[], 'Overall Rating': []})
for i in review_of_product:  
    #length = 0# y is scraped data
    for j, k in i.items():
        for l in k:
            df.loc[count] = [j, l, link_product[length][j], ratings_of_product[length][j]['5 Stars'], ratings_of_product[length][j]['4 Stars'], ratings_of_product[length][j]['3 Stars'], ratings_of_product[length][j]['2 Stars'], ratings_of_product[length][j]['1 Stars'], price_product[length][j], overall_r[length][j] ]
            count+=1
    length += 1
    
df


,Product Name,Review,Link,5 Stars,4 Stars,3 Stars,2 Stars,1 Stars,Price,Overall Rating
0,Apple iPhone 14 (128 GB) - Blue,Not good from iPhone 13,https://www.amazon.in/Apple-iPhone-14-128GB-Bl...,76%,11%,4%,1%,7%,"₹71,999 M.R.P:\n₹79,990",4.5 out of 5
1,Apple iPhone 14 (128 GB) - Blue,all good,https://www.amazon.in/Apple-iPhone-14-128GB-Bl...,76%,11%,4%,1%,7%,"₹71,999 M.R.P:\n₹79,990",4.5 out of 5
2,Apple iPhone 14 (128 GB) - Blue,This iPhone 14 is worth its price. Amazing fea...,https://www.amazon.in/Apple-iPhone-14-128GB-Bl...,76%,11%,4%,1%,7%,"₹71,999 M.R.P:\n₹79,990",4.5 out of 5
3,Apple iPhone 14 (128 GB) - Blue,I updated the iPhone 14 from iPhone 11.I excha...,https://www.amazon.in/Apple-iPhone-14-128GB-Bl...,76%,11%,4%,1%,7%,"₹71,999 M.R.P:\n₹79,990",4.5 out of 5
4,Apple iPhone 14 (128 GB) - Blue,Awesome products,https://www.amazon.in/Apple-iPhone-14-128GB-Bl...,76%,11%,4%,1%,7%,"₹71,999 M.R.P:\n₹79,990",4.5 out of 5
...,...,...,...,...,...,...,...,...,...,...
157,Apple iPhone 14 Pro Max (256 GB) - Deep Purple,Product is used cause I have been using only A...,https://www.amazon.in/Apple-iPhone-256GB-Deep-...,78%,9%,3%,1%,9%,"₹1,49,900",4.5 out of 5
158,Apple iPhone 14 Pro Max (256 GB) - Deep Purple,The price is on higher side but the iPhone com...,https://www.amazon.in/Apple-iPhone-256GB-Deep-...,78%,9%,3%,1%,9%,"₹1,49,900",4.5 out of 5
159,Apple iPhone 14 Pro Max (256 GB) - Deep Purple,"God level performance, best camera quality Vid...",https://www.amazon.in/Apple-iPhone-256GB-Deep-...,78%,9%,3%,1%,9%,"₹1,49,900",4.5 out of 5
160,Apple iPhone 14 Pro Max (256 GB) - Deep Purple,Excellent phone.,https://www.amazon.in/Apple-iPhone-256GB-Deep-...,78%,9%,3%,1%,9%,"₹1,49,900",4.5 out of 5


In [114]:
df

,Product Name,Review,Link,5 Stars,4 Stars,3 Stars,2 Stars,1 Stars,Price,Overall Rating
0,Apple iPhone 14 (128 GB) - Blue,Not good from iPhone 13,https://www.amazon.in/Apple-iPhone-14-128GB-Bl...,76%,11%,4%,1%,7%,"₹71,999 M.R.P:\n₹79,990",4.5 out of 5
1,Apple iPhone 14 (128 GB) - Blue,all good,https://www.amazon.in/Apple-iPhone-14-128GB-Bl...,76%,11%,4%,1%,7%,"₹71,999 M.R.P:\n₹79,990",4.5 out of 5
2,Apple iPhone 14 (128 GB) - Blue,This iPhone 14 is worth its price. Amazing fea...,https://www.amazon.in/Apple-iPhone-14-128GB-Bl...,76%,11%,4%,1%,7%,"₹71,999 M.R.P:\n₹79,990",4.5 out of 5
3,Apple iPhone 14 (128 GB) - Blue,I updated the iPhone 14 from iPhone 11.I excha...,https://www.amazon.in/Apple-iPhone-14-128GB-Bl...,76%,11%,4%,1%,7%,"₹71,999 M.R.P:\n₹79,990",4.5 out of 5
4,Apple iPhone 14 (128 GB) - Blue,Awesome products,https://www.amazon.in/Apple-iPhone-14-128GB-Bl...,76%,11%,4%,1%,7%,"₹71,999 M.R.P:\n₹79,990",4.5 out of 5
...,...,...,...,...,...,...,...,...,...,...
157,Apple iPhone 14 Pro Max (256 GB) - Deep Purple,Product is used cause I have been using only A...,https://www.amazon.in/Apple-iPhone-256GB-Deep-...,78%,9%,3%,1%,9%,"₹1,49,900",4.5 out of 5
158,Apple iPhone 14 Pro Max (256 GB) - Deep Purple,The price is on higher side but the iPhone com...,https://www.amazon.in/Apple-iPhone-256GB-Deep-...,78%,9%,3%,1%,9%,"₹1,49,900",4.5 out of 5
159,Apple iPhone 14 Pro Max (256 GB) - Deep Purple,"God level performance, best camera quality Vid...",https://www.amazon.in/Apple-iPhone-256GB-Deep-...,78%,9%,3%,1%,9%,"₹1,49,900",4.5 out of 5
160,Apple iPhone 14 Pro Max (256 GB) - Deep Purple,Excellent phone.,https://www.amazon.in/Apple-iPhone-256GB-Deep-...,78%,9%,3%,1%,9%,"₹1,49,900",4.5 out of 5


In [117]:
price = df.loc[df['Product Name'] == 'Apple iPhone 14 (128 GB) - Blue', 'Price'].values[0]
price


'₹71,999 M.R.P:\n₹79,990'

In [121]:
# avg = self.df.groupby('Product Name')['sentiment'].mean().sort_values(ascending=False)[:1]
import numpy as np
df['sentiment'] = np.ones(df.shape[0])

In [124]:
avg = df.groupby('Product Name')['sentiment'].mean().sort_values(ascending=False)[:1]

In [125]:
avg.index[0]

'Apple iPhone 13 (128GB) - (Product) RED'

In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import threading


class Amazon:
    def __init__(self, search, limit, page_visit):
        self.search = search
        self.limit = limit
        self.page_visit = page_visit
        self.s = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=self.s)
        self.driver.get('https://www.amazon.in/')
        self.un = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="twotabsearchtextbox"]')))
        self.un.send_keys(search)
        time.sleep(2)
        self.un.send_keys(Keys.ENTER)

    def link_collection(self):
        self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
        first_page_id = self.driver.current_window_handle
        review = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,
                                                                                           'a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')))  # a-size-base.s-underline-text
        reviews = []
        price = []
        for i in review:
            if '₹' not in i.text:
                reviews.append(i)

        for i in review:
            if '₹' in i.text:
                price.append(i.text)

        try:
            title1 = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, 'a-size-base-plus.a-color-base.a-text-normal')))
        except TimeoutException:
            title1 = []

        try:
            title2 = WebDriverWait(self.driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, 'a-size-medium.a-color-base.a-text-normal')))
        except TimeoutException:
            title2 = []

        main_title = title1 if len(title1) > len(title2) else title2

        titles = []
        d = {}
        prices = {}
        for i, j in zip(main_title, reviews):
            x = self.search.lower()
            if ('(Renewed)' not in i.text) or ('WeConnect' not in i.text) or (
                    'Protect+' not in i.text):  # or ('AppleCare' not in i.text)
                d[j] = i.text
                titles.append(i.text)

        for i, j in zip(main_title, price):
            prices[i.text] = j

        time.sleep(1)
        return d, titles, prices

    def scraping(self, d, limit, prices):
        # j = 1
        prod_review = {}
        ratings = {}
        link_of_product = {}
        overall_rating = {}
        for i in d:
            y = d[i]
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            self.driver.switch_to.window(self.driver.window_handles[0])
            # driver.switch_to.window(first_page_id)
            i.click()
            time.sleep(2)
            c = self.driver.window_handles[1]
            self.driver.switch_to.window(c)

            link = self.driver.current_url
            link_of_product[y] = link

            #         #current_page_id = driver.current_window_handle

            #         #driver.switch_to.window(current_page_id)
            #         try:
            #             driver.switch_to.window(driver.window_handles[j])
            #         except IndexError:
            #             driver.switch_to.window(driver.window_handles[0])
            #             j+=1
            #             continue

            time.sleep(6)
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')

            try:
                all_comments = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'a-link-emphasis.a-text-bold')))
                all_comments.click()
                time.sleep(2)
            except TimeoutException:
                pass

            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            try:
                box = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, 'a-dropdown-prompt')))
                for k in box:
                    if k.text == 'All formats':
                        k.click()
            except TimeoutException:
                pass

            try:
                click = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.ID, 'format-type-dropdown_1')))
                click.click()
                time.sleep(3)
            except TimeoutException:
                pass

             # scrapping views
            e = self.driver.page_source
            x = BeautifulSoup(e, 'html.parser')

            # scraping rating
            h = x.find_all('span', {'class': 'a-size-medium a-color-base'})

            clean = ' '
            for g in h:
                cleaned = g.text.strip()
                if g.text.strip() != '':
                    clean += cleaned[:3]
            overall_rating[y] = cleaned.strip()

            ## scraping review

            t = x.find_all('span', {'class': 'a-size-base review-text review-text-content'})
            reviews = set()
            for l in t:
                reviews.add(l.text.strip())
                if len(reviews) == self.limit:
                    break

            # star ratings
            h = x.find_all('td', {'class': 'a-text-right a-nowrap'})
            rating = {}
            counter = 5
            for rat in h:
                rating['{} Stars'.format(counter)] = rat.text.strip()
                counter -= 1

            ratings[y] = rating


            if len(reviews) != self.limit or len(reviews) < self.limit:
                try:
                    n = 0
                    while len(reviews) != self.limit:
                        try:
                            last = WebDriverWait(self.driver, 5).until(
                                EC.presence_of_element_located((By.CLASS_NAME, 'a-disabled.a-last')))
                            x = last.text
                            if x == 'Next page':
                                break
                        except (TimeoutException, StaleElementReferenceException):
                            review1 = reviews
                            next_ = WebDriverWait(self.driver, 10).until(
                                EC.presence_of_element_located((By.CLASS_NAME, 'a-last')))
                            next_.click()
                            time.sleep(4)
                            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
                            e = self.driver.page_source
                            x = BeautifulSoup(e, 'html.parser')
                            t = x.find_all('div', {'class': 'a-size-base review-text review-text-content'})
                            for l in t:
                                reviews.add(l.text.strip())
                                if len(reviews) == self.limit:
                                    break
                            prod_review[y] = reviews
                            if len(reviews) == len(review1):
                                break
                except TimeoutException:
                    pass
            else:
                prod_review[y] = reviews

            prod_review[y] = reviews
            self.driver.close()
            self.driver.switch_to.window(self.driver.window_handles[0])

        return prod_review, ratings, link_of_product, overall_rating

    def run(self, page_visit):
        review_of_product = []
        ratings_of_product = []
        link_product = []
        price_product = []
        overall_r = []

        for i in range(self.page_visit):
            d, titles, prices = self.link_collection()
            print(titles)
            prod, ratings, links, overall = self.scraping(d, self.limit, prices)

            review_of_product.append(prod)
            ratings_of_product.append(ratings)
            link_product.append(links)
            price_product.append(prices)
            overall_r.append(overall)


            self.driver.switch_to.window(self.driver.window_handles[0])
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            time.sleep(2)
            flag = False
            while flag != True:
                try:
                    next_ = WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME,
                                                                                                's-pagination-item.s-pagination-next.s-pagination-button.s-pagination-separator')))
                    next_.click()
                    flag = True
                except (TimeoutException, ElementNotInteractableException, ElementClickInterceptedException):
                    self.driver.refresh()
                    time.sleep(3)
                    pass
        return review_of_product, ratings_of_product, link_product, price_product, overall_r



In [7]:
amazon = Amazon('Oneplus nord', 10, 1)

    # run
review_of_product, ratings_of_product, link_product, price_product, overall_r  = amazon.run(1)

['OnePlus Nord CE 2 Lite 5G (Blue Tide, 6GB RAM, 128GB Storage)', 'OnePlus 11 5G (Eternal Green, 16GB RAM, 256GB Storage)', 'OnePlus Nord CE 3 Lite 5G (Chromatic Gray, 8GB RAM, 128GB Storage)', 'OnePlus Nord CE 3 Lite 5G (Pastel Lime, 8GB RAM, 128GB Storage)', 'OnePlus Nord CE 2 Lite 5G (Black Dusk, 6GB RAM, 128GB Storage)', 'OnePlus Nord 2T 5G (Jade Fog, 8GB RAM, 128GB Storage)', 'OnePlus Nord CE 3 Lite 5G (Pastel Lime, 8GB RAM, 256GB Storage)', 'OnePlus Nord CE 2 Lite 5G (Blue Tide, 6GB RAM, 128GB Storage)', 'OnePlus Nord CE 3 Lite 5G (Chromatic Gray, 8GB RAM, 256GB Storage)', 'OnePlus Nord 2T 5G (Gray Shadow, 8GB RAM, 128GB Storage)', 'OnePlus 11R 5G (Galactic Silver, 16GB RAM, 256GB Storage)', 'Redmi K50i 5G (Quick Silver, 6GB RAM, 128GB Storage) | Flagship Mediatek Dimensity 8100 Processor | 144Hz Liquid FFS Display', '(Renewed) OnePlus Nord 5G Blue Marble, 8GB RAM, 128GB Storage', '(Renewed) OnePlus Nord 2 5G Gray Sierra, 8GB RAM, 128GB Storage', '(Renewed) OnePlus Nord 2 5G Blue

In [8]:
link_product

[{'OnePlus Nord CE 2 Lite 5G (Blue Tide, 6GB RAM, 128GB Storage)': 'https://www.amazon.in/OnePlus-Nord-Lite-128GB-Storage/dp/B09WQYFLRX/ref=sr_1_8?crid=3NPSP7V8Z5EMQ&keywords=Oneplus%2Bnord&qid=1683261413&sprefix=oneplus%2Bnord%2Caps%2C450&sr=8-8&th=1',
  'OnePlus 11 5G (Eternal Green, 16GB RAM, 256GB Storage)': 'https://www.amazon.in/OnePlus-Eternal-Green-256GB-Storage/dp/B0BQJM2PXW/ref=sr_1_2_sspa?crid=3NPSP7V8Z5EMQ&keywords=Oneplus%2Bnord&qid=1683261413&sprefix=oneplus%2Bnord%2Caps%2C450&sr=8-2-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&th=1',
  'OnePlus Nord CE 3 Lite 5G (Chromatic Gray, 8GB RAM, 128GB Storage)': 'https://www.amazon.in/OnePlus-Nord-Chromatic-128GB-Storage/dp/B0BY8MCQ9S/ref=sr_1_3?crid=3NPSP7V8Z5EMQ&keywords=Oneplus%2Bnord&qid=1683261413&sprefix=oneplus%2Bnord%2Caps%2C450&sr=8-3&th=1',
  'OnePlus Nord CE 3 Lite 5G (Pastel Lime, 8GB RAM, 128GB Storage)': 'https://www.amazon.in/OnePlus-Nord-Pastel-128GB-Storage/dp/B0BY8JZ22K/ref=sr_1_4?crid=3NPSP7V8Z5EMQ&keywords=Oneplus%2B